In [92]:
#Find interesting things
import sqlite3
import urllib.parse
import nltk

In [148]:
conn = sqlite3.connect('db.db')
airline_ids = [1126873599372156900, 1024338385404682200, 211830847, 56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406, 38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]

In [107]:
conn.execute("CREATE INDEX in_rp_t_st_id ON retweets (in_reply_to_status_id);")
conn.execute("CREATE INDEX ent_ment_id ON ent_mentions (user_id);")

In [113]:
conn.execute("CREATE INDEX ret_stat ON tweets (retweeted_status);")

In [149]:
#Get retweets only by airlines
query_1 = """
select user, count(user) as user_count
from tweets
where user NOT IN ({0})
and text = 'null'
and retweeted_status != 'null'
group by user
order by user_count desc
LIMIT 250;
""".format(','.join((str(x) for x in airline_ids)))
for row in conn.execute(query_1):
    user_id = row[0]
    amount = row[1]
    #Get all airline mentions
    data = conn.execute("select name, count(id) from users where id = '{0}' group by id order by timestamp desc"
                        .format(user_id)).fetchone()
    name = urllib.parse.unquote_plus(data[0])
    status_count = data[1]
    print("ID: {0}, Name: {1} with status count {2}, {3} {4} retweets"
          .format(user_id, name, status_count, amount / status_count, amount))
    for airline in airline_ids:
        query_2 ="""
        select count(id) from tweets where
        id in (SELECT id FROM ent_mentions where user_id = '{0}')
        and
        user = '{1}'
        """
        airline_numbers = conn.execute(query_2.format(str(airline), user_id)).fetchone()[0]
        print(airline, airline_numbers)
   


ID: 875637231112683500, Name: PlaneSpottingIsCool with status count 8962, 1.0 8962 retweets
1126873599372156900 0
1024338385404682200 0
211830847 0
56377143 559
106062176 306
18332190 900
22536055 550
124476322 586
26223583 3
2182373406 0
38676903 319
1542862735 202
253340062 191
218730857 197
45621423 224
20626359 376
ID: 1137383922843770900, Name: Shield with status count 8508, 0.9998824635637047 8507 retweets
1126873599372156900 0
1024338385404682200 1
211830847 1
56377143 315
106062176 189
18332190 1905
22536055 192
124476322 287
26223583 0
2182373406 0
38676903 1082
1542862735 1002
253340062 66
218730857 36
45621423 113
20626359 34
ID: 799303160452018200, Name: British Airways SUCK with status count 7210, 0.7988904299583911 5760 retweets
1126873599372156900 2
1024338385404682200 0
211830847 50
56377143 9
106062176 4
18332190 6748
22536055 60
124476322 10
26223583 0
2182373406 0
38676903 34
1542862735 30
253340062 1
218730857 5
45621423 11
20626359 42
ID: 3385729313, Name: Twintair

KeyboardInterrupt: 

In [133]:
#Get retweets all
query_1 = """
select user, count(user) as user_count
from retweets
where user NOT IN ({0})
group by user
order by user_count desc
LIMIT 250;
""".format(','.join((str(x) for x in airline_ids)))
for row in conn.execute(query_1):
    user_id = row[0]
    amount = row[1]
    #Get all airline mentions
    data = conn.execute("select name, count(id) from users where id = '{0}' group by id order by timestamp desc"
                        .format(user_id)).fetchone()
    name = urllib.parse.unquote_plus(data[0])
    status_count = data[1]
    print("ID: {0}, Name: {1} with status count {2}, {3} {4} retweets"
          .format(user_id, name, status_count, amount / status_count, amount))
    query_2 ="""
    select user_id, count(user_id) as user_count from ent_mentions where id
    in (select id from retweets where
    user = '{1}')
    group by user_id
    order by user_count desc
    limit 10;
    """
    for a in conn.execute(query_2.format(str(user_id), user_id)):
        print("{0}>>>{1}".format(user_id, a))
   


ID: 1449525326, Name: Islam with status count 79, 1110.26582278481 87711 retweets
1449525326>>>(1542862735, 387)
1449525326>>>(17842366, 290)
1449525326>>>(1139432945092112400, 12)
1449525326>>>(976029015667630100, 9)
1449525326>>>(797779478961356800, 9)
1449525326>>>(1155598416740851700, 4)
1449525326>>>(1102280513526657000, 4)
1449525326>>>(3375577991, 4)
1449525326>>>(2947253139, 4)
1449525326>>>(394862725, 4)
ID: 1001079229864398800, Name: luna with status count 2, 19407.5 38815 retweets
ID: 1399161414, Name: CHERSAN SRISATJANG✈️ with status count 13, 2492.6153846153848 32404 retweets
1399161414>>>(914744345965756400, 8)
1399161414>>>(883957981389307900, 8)
1399161414>>>(91172195, 8)
ID: 427537312, Name: Kareem with status count 2, 15864.0 31728 retweets
ID: 14813703, Name: Matthew Harris with status count 2, 12310.0 24620 retweets
14813703>>>(1542862735, 84042)
14813703>>>(123908101, 84042)
14813703>>>(27067896, 84042)
14813703>>>(38676903, 1207)
ID: 3716026639, Name: ㅎㅕㄴ모양ㅊㅓ-메인트윗

883957981389307900>>>(202923584, 2)
883957981389307900>>>(124476322, 2)
883957981389307900>>>(95584582, 2)
ID: 878247600096510000, Name: Candace Owens with status count 4, 1380.5 5522 retweets
878247600096510000>>>(22536055, 5860)
878247600096510000>>>(1179417193454420000, 13)
878247600096510000>>>(20873929, 11)
ID: 136004952, Name: Andrew Neil with status count 4, 1337.5 5350 retweets
136004952>>>(18332190, 5)
ID: 2280470022, Name: AIRLIVE with status count 335, 15.967164179104477 5349 retweets
2280470022>>>(18332190, 585)
2280470022>>>(106062176, 317)
2280470022>>>(38676903, 249)
2280470022>>>(463932884, 211)
2280470022>>>(218730857, 175)
2280470022>>>(22536055, 140)
2280470022>>>(124476322, 68)
2280470022>>>(1542862735, 43)
2280470022>>>(15425377, 15)
2280470022>>>(1443360055, 12)
ID: 950183205285527600, Name: Jon from Derby 🏴󠁧󠁢󠁷󠁬󠁳󠁿🇬🇧🇪🇺 🐏#FBPE with status count 40, 132.825 5313 retweets
950183205285527600>>>(1466783923, 28)
950183205285527600>>>(949692936614285300, 25)
9501832052855

ID: 3226842193, Name: สาระการบินน่ารู้🛩✨™ with status count 229, 15.117903930131005 3462 retweets
3226842193>>>(60361925, 24)
3226842193>>>(954195006121627600, 5)
3226842193>>>(95584582, 5)
3226842193>>>(56377143, 2)
ID: 36042554, Name: The Royal Family with status count 16, 210.75 3372 retweets
36042554>>>(18332190, 2949)
ID: 42421075, Name: rizarul sorry penad with status count 5, 669.8 3349 retweets
ID: 191881312, Name: Alex Macheras with status count 221, 14.846153846153847 3281 retweets
191881312>>>(18332190, 185)
191881312>>>(20626359, 107)
191881312>>>(15425377, 71)
191881312>>>(124476322, 64)
191881312>>>(76067316, 46)
191881312>>>(22536055, 44)
191881312>>>(45621423, 43)
191881312>>>(38676903, 31)
191881312>>>(106062176, 18)
191881312>>>(20823928, 13)
ID: 2546037755, Name: Airplane Pictures ✈ with status count 16, 203.1875 3251 retweets
ID: 805126491528892400, Name: Monica Carol with status count 1, 3215.0 3215 retweets
ID: 2786230659, Name: abraar 🇲🇺 with status count 6, 535.

3385729313>>>(40205617, 173)
3385729313>>>(22090282, 134)
3385729313>>>(15425377, 122)
3385729313>>>(138552413, 107)
3385729313>>>(171469656, 98)
3385729313>>>(2244049410, 88)
3385729313>>>(99984408, 81)
3385729313>>>(20814874, 67)
3385729313>>>(434844586, 63)
3385729313>>>(3044797241, 54)
ID: 516213072, Name: Crisologo Caceres with status count 1, 2322.0 2322 retweets
ID: 958694618, Name: Airport Workers Utd with status count 50, 45.44 2272 retweets
958694618>>>(22536055, 2280)
958694618>>>(342803091, 2232)
958694618>>>(5920532, 33)
958694618>>>(249768963, 18)
958694618>>>(7212562, 9)
958694618>>>(361321755, 6)
958694618>>>(260907612, 3)
ID: 36109768, Name: 𝙀𝙣𝙧𝙞𝙦𝙪𝙚 𝙂𝙖𝙡𝙫á𝙣 𝙊𝙘𝙝𝙤𝙖 with status count 7, 320.85714285714283 2246 retweets
36109768>>>(124476322, 2459)
ID: 301308940, Name: Rubén Sánchez with status count 43, 51.46511627906977 2213 retweets
301308940>>>(8077212, 445)
301308940>>>(202372417, 362)
301308940>>>(925676922838929400, 228)
301308940>>>(1216727603689771000, 134)
3013089

ID: 47438401, Name: Richard Engel with status count 1, 1814.0 1814 retweets
ID: 1957125536, Name: Old Holborn ✘ with status count 21, 86.19047619047619 1810 retweets
1957125536>>>(1245699895, 7)
1957125536>>>(2985942701, 3)
1957125536>>>(603783854, 3)
1957125536>>>(20973388, 3)
ID: 61189317, Name: Lord Inario with status count 2, 900.0 1800 retweets
ID: 739525969111830500, Name: BIG JET TV with status count 161, 10.850931677018634 1747 retweets
739525969111830500>>>(18332190, 1823)
739525969111830500>>>(587397179, 1413)
739525969111830500>>>(20823928, 209)
739525969111830500>>>(15425377, 201)
739525969111830500>>>(20626359, 195)
739525969111830500>>>(46366572, 100)
739525969111830500>>>(868181886958530600, 79)
739525969111830500>>>(19913832, 58)
739525969111830500>>>(22536055, 52)
739525969111830500>>>(59745629, 45)
ID: 1033551266155098100, Name: 柱島の東北ずん子総統 with status count 23, 74.08695652173913 1704 retweets
ID: 96634851, Name: Andrea Blanford with status count 6, 284.0 1704 retweets

ID: 3056307455, Name: Zarah Sultana MP with status count 2, 701.0 1402 retweets
ID: 6135622, Name: DW Español with status count 25, 56.0 1400 retweets
ID: 887174165836423200, Name: AK Entertainments with status count 1, 1377.0 1377 retweets
ID: 707197958333337600, Name: Julie Owen Moylan with status count 6, 227.16666666666666 1363 retweets
ID: 909959658101661700, Name: คูรึมแลนด์ | 🐶💕 with status count 1, 1361.0 1361 retweets
ID: 42479903, Name: Ecuadorinmediato with status count 9, 151.22222222222223 1361 retweets
42479903>>>(300279484, 31)
ID: 1127407858608304100, Name: ryujehong with status count 1, 1358.0 1358 retweets
1127407858608304100>>>(22536055, 2448)
ID: 69250474, Name: Confederación USO with status count 299, 4.521739130434782 1352 retweets
69250474>>>(1101556894429253600, 155)
69250474>>>(1012067223358328800, 55)
69250474>>>(217352125, 39)
69250474>>>(249616596, 36)
69250474>>>(273114768, 35)
69250474>>>(955794446997745700, 33)
69250474>>>(874167384181018600, 32)
69250474